In [1]:
import requests
import pandas as pd

import json

## Mapeamento ISIC e CNAE

### Carregar tabelas do comexstat de ISIC e NCM/SH para o mapeamento com ISIC

In [2]:
ncm_isic = pd.read_csv('./dados/comex/NCM_ISIC (1).csv',sep=';',dtype={'CO_ISIC_CLASSE': str, 'NO_ISIC_CLASSE': str}    )
ncm = pd.read_csv('./dados/comex/NCM (2).csv',sep=';',dtype={'CO_SH6': str, 'CO_ISIC_CLASSE': str})

### Carregar a tabela de mapeamento de ISIC para CNAE do IBGE

In [3]:
mapeamento_isic = pd.read_excel('./dados/mapeamento/mapeamento_isic_cnae.xlsx', dtype=str).drop(columns=['Unnamed: 3','Observações'])

##### Fazer a primeira tentativa usando códigos ISIC a 3 dígitos

In [4]:
mapeamento_isic = mapeamento_isic[mapeamento_isic['código ISIC'].str.len() == 3]
mapeamento_isic.tail()

,código ISIC,denominação ISIC,código CNAE,denominação CNAE
1171,960,Otras actividades de servicios personales,96.0,Outras atividades de serviços pessoais
1178,970,Actividades de los hogares como empleadores de...,97.0,Serviços domésticos
1181,981,Actividades no diferenciadas de los hogares co...,NaN,NaN
1183,982,Actividades no diferenciadas de los hogares co...,NaN,NaN
1187,990,Actividades de organizaciones y órganos extrat...,99.0,Organismos internacionais e outras instituiçõe...


#### Eliminar códigos ISIC sem correspondência

In [5]:
mapeamento_isic = mapeamento_isic.dropna()

In [6]:
mapeamento_isic['CNAE2d'] = mapeamento_isic['código CNAE'].str[0:2]
mapeamento_isic['ISIC2d'] = mapeamento_isic['código ISIC'].str[0:2]

#### Verificar se há códigos ISIC a dois dígitos que correspondem a mais de uma cnae a dois dígitos

In [7]:
mapeamento_isic.groupby('ISIC2d')[['CNAE2d']].nunique().sort_values(by='CNAE2d', ascending=False).head(10)

,CNAE2d
ISIC2d,
19,2
22,2
16,2
20,2
88,2
70,2
58,2
08,1
07,1


In [8]:
mapeamento_isic.query('ISIC2d.isin(["58","70","88"])')

,código ISIC,denominação ISIC,código CNAE,denominação CNAE,CNAE2d,ISIC2d
822,581,Edición de libros y publicaciones periódicas y...,58.1,"Edição de livros, jornais, revistas e outras a...",58,58
823,581,Edición de libros y publicaciones periódicas y...,58.2,"Edição integrada à impressão de livros, jornai...",58,58
834,582,Edición de programas informáticos,62.0,Atividades dos serviços de tecnologia da infor...,62,58
955,701,Actividades de oficinas principales,64.6,Atividades de sociedades de participação,64,70
956,701,Actividades de oficinas principales,70.1,Sedes de empresas e unidades administrativas l...,70,70
960,702,Actividades de consultoría de gestión,70.2,Atividades de consultoria em gestão empresarial,70,70
1117,881,Actividades de asistencia social sin alojamien...,88.0,Serviços de assistência social sem alojamento,88,88
1119,889,Otras actividades de aistencia social sin aloj...,85.1,Educação infantil e ensino fundamental,85,88
1120,889,Otros servicios sociales sin alojamiento n.c.p.,88.0,Serviços de assistência social sem alojamento,88,88


#### Códigos a 2 dígitos ISIC que estão relacionados a mais de um código CNAE a 2 dígitos sempre estão vinculados a dígitos idênticos mais um outro. Nesses casos, se considerará apenas os dois dígitos idênticos, descartando o segundo

In [9]:
mapeamento_isic.query('`CNAE2d`.isin(["15"])')

,código ISIC,denominação ISIC,código CNAE,denominação CNAE,CNAE2d,ISIC2d
213,151,Curtido y adobo de cueros; fabricación de male...,15.1,Curtimento e outras preparações de couro,15,15
214,151,Curtido y adobo de cueros; fabricación de male...,15.2,Fabricação de artigos para viagem e de artefat...,15,15
218,152,Fabricación de calzado,15.3,Fabricação de calçados,15,15
219,152,Fabricación de calzado,15.4,"Fabricação de partes para calçados, de qualque...",15,15
229,162,"Fabricación de productos de madera, corcho, pa...",15.4,"Fabricação de partes para calçados, de qualque...",15,16
315,221,Fabricación de productos de caucho,15.4,"Fabricação de partes para calçados, de qualque...",15,22
321,222,Fabricación de productos de plástico,15.4,"Fabricação de partes para calçados, de qualque...",15,22


Decidiu-se por usar os dois primeiros dígitos do ISIC como aproximação da CNAE, quando a correspondência a 3 dígitos não é possível, por haver mapeamento a mais de uma CNAE vinculadao ao código ISIC a 3 dígitos. Ver seção de mapeamento ISIC x CNAE x PIA

Remover mapeamentos CNAE ISIC para códigos com os dois primeiros dígitos divergentes.

In [10]:
mapeamento_isic = mapeamento_isic.query('CNAE2d == ISIC2d')

In [11]:

tarifas = pd.read_excel('./dados/comex/impacto_empresas/tarifas_totais_232.xlsx')

In [12]:
tarifas['Articles of Civil Aircraft only'].fillna('',inplace=True)

/tmp/ipykernel_1978/2934317571.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tarifas['Articles of Civil Aircraft only'].fillna('',inplace=True)


In [13]:
tarifas.head(2)

,SH6,Instrumento,Tarifa,Vigência,Tarifa Desc,Articles of Civil Aircraft only
0,490110,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,
1,490191,Ordem 30/07,0.5,Vigente,Tarifa de 10+40%,


In [14]:
mapeamento_isic.head(2)

,código ISIC,denominação ISIC,código CNAE,denominação CNAE,CNAE2d,ISIC2d
2,011,Cultivo de plantas no perenes,01.1,Produção de lavouras temporárias,01,01
3,011,Cultivo de plantas no perenes,01.2,Horticultura e floricultura,01,01


In [15]:
ncm = pd.read_csv('./dados/comex/NCM (2).csv',sep=';',dtype={'CO_SH6': str, 'CO_ISIC_CLASSE': str})
### Carregar a tabela de mapeamento de ISIC para CNAE do IBGE
mapeamento_isic = pd.read_excel('./dados/mapeamento/mapeamento_isic_cnae.xlsx', dtype=str).drop(columns=['Unnamed: 3','Observações'])
##### Fazer a primeira tentativa usando códigos ISIC a 3 dígitos
mapeamento_isic = mapeamento_isic[mapeamento_isic['código ISIC'].str.len() == 3]
mapeamento_isic.tail()
#### Eliminar códigos ISIC sem correspondência
mapeamento_isic = mapeamento_isic.dropna()
mapeamento_isic['CNAE2d'] = mapeamento_isic['código CNAE'].str[0:2]
mapeamento_isic['ISIC2d'] = mapeamento_isic['código ISIC'].str[0:2]
#### Verificar se há códigos ISIC a dois dígitos que correspondem a mais de uma cnae a dois dígitos
mapeamento_isic.groupby('ISIC2d')[['CNAE2d']].nunique().sort_values(by='CNAE2d', ascending=False).head(10)
#### Códigos a 2 dígitos ISIC que estão relacionados a mais de um código CNAE a 2 dígitos sempre estão vinculados a dígitos idênticos mais um outro. Nesses casos, se considerará apenas os dois dígitos idênticos, descartando o segundo
mapeamento_isic = mapeamento_isic.query('CNAE2d == ISIC2d')
### Remover exportações com valor 0 e mapear exportações com código ISIC a 4 dígitos

mapeamento_isic['cnae'] = mapeamento_isic['código CNAE'].str.replace('.','', regex=False)
#### Verificar as situações em que um ISIC a 3 dígitos é mapeado para mais de uma CNAE


In [16]:
ncm['CO_ISIC_CLASSE3d'] = ncm['CO_ISIC_CLASSE'].str[0:3]

mapeamento_isic_ncm = mapeamento_isic.merge(ncm[['CO_ISIC_CLASSE3d','CO_SH6']].drop_duplicates(), left_on='código ISIC', right_on='CO_ISIC_CLASSE3d', how='inner')

In [17]:
mapeamento_isic_ncm.shape

(14482, 9)

In [18]:
lista_isic_unica_cnae = mapeamento_isic_ncm.groupby('código ISIC')[['cnae']].nunique().query('cnae == 1').reset_index()['código ISIC'].tolist()
mapeamento_isic3d = pd.read_excel('./dados/mapeamento/mapeamento_isic_cnae.xlsx', dtype=str).drop(columns=['Unnamed: 3','Observações'])
mapeamento_isic3d['código ISIC'] = mapeamento_isic3d['código ISIC'].str.replace('.','', regex=False)
mapeamento_isic3d['código CNAE'] = mapeamento_isic3d['código CNAE'].str.replace('.','', regex=False)

mapeamento_isic3d = mapeamento_isic3d[mapeamento_isic3d['código ISIC'].str.len() == 3]

In [19]:
mapeamento_isic3d.head(2)

,código ISIC,denominação ISIC,código CNAE,denominação CNAE
2,011,Cultivo de plantas no perenes,011,Produção de lavouras temporárias
3,011,Cultivo de plantas no perenes,012,Horticultura e floricultura


In [20]:
mapeamento_isic = pd.read_excel('./dados/mapeamento/mapeamento_isic_cnae.xlsx', dtype=str).drop(columns=['Unnamed: 3','Observações'])
mapeamento_isic['código ISIC'] = mapeamento_isic['código ISIC'].str.replace('.','', regex=False)
mapeamento_isic['código CNAE'] = mapeamento_isic['código CNAE'].str.replace('.','', regex=False)

mapeamento_isic = mapeamento_isic[mapeamento_isic['código ISIC'].str.len() == 2]
mapeamento_isic = mapeamento_isic[['código CNAE','denominação CNAE']]
mapeamento_isic['denominação CNAE'] = mapeamento_isic['denominação CNAE'].str.capitalize()

mapeamento_isic.head(2)

,código CNAE,denominação CNAE
1,01,"Agricultura, pecuária e serviços relacionados"
68,02,Produção florestal


In [21]:
mapeamento_isic = mapeamento_isic.drop_duplicates()

In [22]:
mapeamento_isic_ncm.head(2)

,código ISIC,denominação ISIC,código CNAE,denominação CNAE,CNAE2d,ISIC2d,cnae,CO_ISIC_CLASSE3d,CO_SH6
0,011,Cultivo de plantas no perenes,01.1,Produção de lavouras temporárias,01,01,011,011,520100
1,011,Cultivo de plantas no perenes,01.1,Produção de lavouras temporárias,01,01,011,011,070956


In [23]:
#Para cada código cnae que possuir comprimento 2 dígitos, substituir o campo 'denominação CNAE' pelo valor correspondente em mapeamento_isic, sem alterar o valor nos casos em que esse campo possua 3 dígitos
#Criar máscara para códigos CNAE de 2 dígitos
mask_2_digitos = mapeamento_isic_ncm['cnae'].str.len() == 2

# Verificar se o mapeamento existe antes de aplicar
if mask_2_digitos.any():
    # Fazer o mapeamento
    mapeamento_dict = mapeamento_isic.set_index('código CNAE')['denominação CNAE'].to_dict()
    mapeamento_isic_ncm.loc[mask_2_digitos, 'denominação CNAE'] = mapeamento_isic_ncm.loc[mask_2_digitos, 'cnae'].map(mapeamento_dict)
    
    # Verificar se ainda há valores nulos após o mapeamento
    nulos_restantes = mapeamento_isic_ncm.loc[mask_2_digitos, 'denominação CNAE'].isna().sum()
    if nulos_restantes > 0:
        print(f"Atenção: {nulos_restantes} códigos CNAE de 2 dígitos não foram mapeados")
mapeamento_isic_ncm = mapeamento_isic_ncm[['cnae', 'denominação CNAE','CO_SH6']].drop_duplicates()
mapeamento_isic_ncm.head(2)

,cnae,denominação CNAE,CO_SH6
0,011,Produção de lavouras temporárias,520100
1,011,Produção de lavouras temporárias,070956


In [24]:
lista_empresas = pd.read_excel('./dados/comex/Lista de empresas.xlsx')

In [25]:
tarifas[['SH6']].drop_duplicates().shape

(6617, 1)

In [26]:
tarifas['SH6'] = tarifas['SH6'].astype(str).str.zfill(6)

In [27]:
lista_sh6 = tarifas.merge(mapeamento_isic_ncm,left_on='SH6',right_on='CO_SH6',how='outer',indicator=True).query('_merge =="left_only"')['SH6'].to_list()

In [28]:
ncm.query('CO_SH6.isin(@lista_sh6)')['CO_ISIC_CLASSE'].drop_duplicates()

44    8999
Name: CO_ISIC_CLASSE, dtype: object

In [29]:
ncm_isic.query('CO_ISIC_CLASSE =="8999"')

,CO_ISIC_CLASSE,NO_ISIC_CLASSE,NO_ISIC_CLASSE_ING,NO_ISIC_CLASSE_ESP,CO_ISIC_GRUPO,NO_ISIC_GRUPO,NO_ISIC_GRUPO_ING,NO_ISIC_GRUPO_ESP,CO_ISIC_DIVISAO,NO_ISIC_DIVISAO,NO_ISIC_DIVISAO_ING,NO_ISIC_DIVISAO_ESP,CO_ISIC_SECAO,NO_ISIC_SECAO,NO_ISIC_SECAO_ING,NO_ISIC_SECAO_ESP
239,8999,Não classificado,Not Classified,Não classificado,899,Não classificado,Not Classified,Não classificado,89,Não classificado,Not Classified,Não classificado,D,Outros Produtos,Other products,Outros Produtos


In [94]:
tarifas.merge(mapeamento_isic_ncm,left_on='SH6',right_on='CO_SH6',how='inner').query('cnae == "011"').query('`Tarifa Desc`=="Adicional de até 10%"')



,SH6,Instrumento,Tarifa,Vigência,Tarifa Desc,Articles of Civil Aircraft only,cnae,denominação CNAE,CO_SH6
8420,080121,Exclusão Ordem 30/07,0.1,Vigente,Adicional de até 10%,,011,Produção de lavouras temporárias,080121


In [30]:
tarifas_cnae = tarifas.merge(mapeamento_isic_ncm,left_on='SH6',right_on='CO_SH6',how='inner')[['cnae','denominação CNAE','Instrumento','Tarifa','Tarifa Desc','Articles of Civil Aircraft only']].drop_duplicates()

In [31]:
#Fazer o merge entre cnae de lista empresas e o maior número de dígitos possíveis de tarifas_cnae



In [32]:
tarifas_cnae.head(2)

,cnae,denominação CNAE,Instrumento,Tarifa,Tarifa Desc,Articles of Civil Aircraft only
0,581,"Edição de livros, jornais, revistas e outras a...",Ordem 30/07,0.5,Tarifa de 10+40%,
1,582,"Edição integrada à impressão de livros, jornai...",Ordem 30/07,0.5,Tarifa de 10+40%,


In [ ]:
tarifas_cnae_maior_tarifa = tarifas_cnae.loc[tarifas_cnae.groupby(['cnae','denominação CNAE'])['tarifa'].idxmax()].reset_index(drop=True)


In [29]:
lista_empresas.head(2)

,co_oper,no_razao_soc,co_cnae,no_cnae
0,43201151001001,CUMMINS BRASIL LIMITADA,4530,Comércio de peças e acessórios para veículos a...
1,3622151000170,NEVESCO IMPORTACAO EXPORTACAO LTDA,4651,"Comércio atacadista de computadores, periféric..."


In [37]:
lista_empresas['co_cnae'] = lista_empresas['co_cnae'].astype(str).str.zfill(4)

In [39]:
df_merged = pd.DataFrame()
tarifas_cnae['comprimento'] = tarifas_cnae['cnae'].str.len()

tarifas_cnae = tarifas_cnae.sort_values(by='comprimento',ascending=False)
# Guardar índices já encontrados
matched_idx = set()

for length in sorted(tarifas_cnae['comprimento'].unique(), reverse=True):
    subset_df2 = tarifas_cnae[tarifas_cnae['comprimento'] == length].copy()
    temp = lista_empresas[~lista_empresas.index.isin(matched_idx)].copy()  # só pega os que ainda não casaram
    temp['prefix'] = temp['co_cnae'].str[:length]
    
    merged = temp.merge(subset_df2, left_on='prefix', right_on='cnae', how='inner')
    df_merged = pd.concat([df_merged, merged], ignore_index=True)
    
    matched_idx.update(merged.index)  # marca como já encontrado

# Resultado final
df_merged = df_merged.drop(columns=['comprimento', 'prefix'])
print(df_merged)

              co_oper                                       no_razao_soc  \
0      61403218000181                             MERSEN DO BRASIL LTDA.   
1      61403218000181                             MERSEN DO BRASIL LTDA.   
2        805870000642                     HYVA DO BRASIL HIDRAULICA LTDA   
3        805870000642                     HYVA DO BRASIL HIDRAULICA LTDA   
4        805870000642                     HYVA DO BRASIL HIDRAULICA LTDA   
...               ...                                                ...   
11576    400634000131  GRAPHIC PACKAGING INTERNATIONAL DO BRASIL - EM...   
11577    400634000131  GRAPHIC PACKAGING INTERNATIONAL DO BRASIL - EM...   
11578    400634000131  GRAPHIC PACKAGING INTERNATIONAL DO BRASIL - EM...   
11579  14162300000181  FIBRAS BIGUACU FABRICACAO E COMERCIO DE EMBARC...   
11580     99817000169  RABBIT INDUSTRIA E COMERCIO DE PRODUTOS DE HIG...   

      co_cnae                                            no_cnae cnae  \
0        2710 

In [97]:
df_merged.shape

(11581, 10)

In [79]:
setores_empresas = df_merged[['cnae','denominação CNAE','Instrumento','Tarifa','Tarifa Desc','co_oper','no_razao_soc']].drop_duplicates().copy()
setores_aviacao_potencial = df_merged.query('`Articles of Civil Aircraft only` == "*"')[['cnae','Articles of Civil Aircraft only']].drop_duplicates().copy()

In [98]:
setores_empresas.shape

(7875, 7)

In [80]:
setores_aviacao_potencial = setores_aviacao_potencial.rename(columns={'cnae' : 'Código CNAE', 'Articles of Civil Aircraft only' : 'Redução de tarifa a 10%(base) caso de produtos relacionados à aviação civil'})

In [81]:
setores_aviacao_potencial['Redução de tarifa a 10%(base) caso de produtos relacionados à aviação civil'] = 'Sim'

In [82]:
setores_empresas = setores_empresas.rename(columns={'co_oper':'CNPJ','no_razao_soc':'Razão Social','cnae' : 'Código CNAE','denominação CNAE' : 'Descrição CNAE'})

In [90]:
setores_empresas_aviacao = setores_empresas.merge(setores_aviacao_potencial, on='Código CNAE', how='left').fillna('Não')

In [ ]:
df_pivot = setores_empresas_aviacao.pivot_table(
    index=['Código CNAE', 'Descrição CNAE', 'CNPJ', 'Razão Social',
           'Redução de tarifa a 10%(base) caso de produtos relacionados à aviação civil'],
    columns='Instrumento',
    values='Tarifa Desc',
    aggfunc='first'  # caso haja mais de uma ocorrência, pega a primeira
).reset_index()

# Remove o nome da coluna gerado pelo pivot
df_pivot.columns.name = None

df_pivot = df_pivot.fillna('Não')

In [101]:
df_pivot['CNPJ'] = df_pivot['CNPJ'].astype(str).str.zfill(14)

In [102]:
df_pivot.to_excel('./Outputs/final/setores_empresas/priorizacao_empresas_exportadoras_setores.xlsx', index=False)   